# **Cloud-based PE Malware Detection API:**

## **Task 2: Deploy your model on the cloud:**

In this task, you will be using Amazon Sagemaker to deploy your model on the cloud, and create an endpoint (~ API) so that other applications can make use
of the model. While this might sound very complicated, you will find that it is actually quite simple to deploy models using Sagemaker. 

In [ ]:
#setting up
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import keras
from keras.models import model_from_json
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

Using TensorFlow backend.


Create a directory and upload the saved model and its weights to it.

In [ ]:
!mkdir keras_model

mkdir: cannot create directory ‘keras_model’: File exists


In [ ]:
#loading the model
json_file = open('keras_model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects={"GlorotUniform": tf.keras.initializers.glorot_uniform})




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
#loading the model weights
loaded_model.load_weights('keras_model/weights.h5')
print("Loaded model from disk")







Loaded model from disk


Create a directory export/servo/1 for the deployment of the model to cloud.

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed - see notes for the next section

model_version = '1'
export_dir = 'export/Servo/' + model_version
export_dir

'export/Servo/1'

In [ ]:

import shutil
shutil.rmtree(export_dir)

In [ ]:
# Build the Protocol Buffer SavedModel at 'export_dir'
build = builder.SavedModelBuilder(export_dir)

In [ ]:
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [ ]:

from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    build.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    build.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [ ]:
!ls export/Servo/variables

ls: cannot access export/Servo/variables: No such file or directory


In [ ]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [ ]:
!touch train.py

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium')

-----------------!CPU times: user 512 ms, sys: 29.5 ms, total: 541 ms
Wall time: 8min 32s


In [ ]:
predictor.endpoint

'sagemaker-tensorflow-2020-04-28-17-18-22-025'

In [ ]:
endpoint_name = 'sagemaker-tensorflow-2020-04-28-17-18-22-025'

The enpoint is created successfully.